In [236]:
from collections import defaultdict

In [237]:
from itertools import islice
with open("wikicorpus_01") as myfile:
    head = list(islice(myfile, 150000))

In [238]:
sent = []
for s in head:
    sent.append(s.split())

In [239]:
words = []
sentences = []
for s in sent:
    if len(s) == 4 and s[0] not in {'(', ')', ','} and '[' not in s[1]:
        words.append((s[0], s[1], s[2], s[3]))
        #if 'endofarticle' in s[1]:
        #    sentences.append(words)
        #    words = []

In [240]:
words[:4]

[('Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'),
 ('La', 'el', 'DA0FS0', '0'),
 ('Plata', 'plata', 'NP00000', '10501473'),
 ('Buenos_Aires', 'buenos_aires', 'NP00000', '0')]

In [241]:
#Creamos un diccionario con las palabras y su cantidad de apariciones
count = dict()
for fst,snd,trd, fth in words:
    if fst not in count:
        count[fst] = 1
    else:
        count[fst] += 1

In [242]:
#Tupla con (word,lemma,PoS tag, synset)
aux = []
check = []   #Lista de listas de cada oración
for fst,snd,trd, fth in words:
    aux.append((fst,snd,trd, fth))
    if fst == '.':
        check.append(aux)
        aux = []

In [243]:
check[:2]

[[('Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'),
  ('La', 'el', 'DA0FS0', '0'),
  ('Plata', 'plata', 'NP00000', '10501473'),
  ('Buenos_Aires', 'buenos_aires', 'NP00000', '0'),
  ('Argentina', 'argentina', 'NP00000', '02035234'),
  ('es', 'ser', 'VSIP3S0', '01775973'),
  ('un', 'uno', 'DI0MS0', '0'),
  ('futbolista', 'futbolista', 'NCCS000', '0'),
  ('argentino', 'argentino', 'AQ0MS0', '00278090'),
  ('.', '.', 'Fp', '0')],
 [('Juega', 'jugar', 'VMM02S0', '00727813'),
  ('de', 'de', 'SPS00', '0'),
  ('delantero', 'delantero', 'NCMS000', '00466114'),
  ('y', 'y', 'CC', '0'),
  ('su', 'su', 'DP3CS0', '0'),
  ('equipo', 'equipo', 'NCMS000', '06093198'),
  ('actual', 'actual', 'AQ0CS0', '01667781'),
  ('es', 'ser', 'VSIP3S0', '01775973'),
  ('Gimnasia', 'gimnasia', 'NP00000', '00275488'),
  ('y', 'y', 'CC', '0'),
  ('Esgrima_La_Plata_de_la_Primera_División_de_Argentina',
   'esgrima_la_plata_de_la_primera_división_de_argentina',
   'NP00000',
   '0'),
  ('.', '.'

In [244]:
#Diccionario con todas las palabras anteriores a una palabra (ventana de 1) --> dicc_backward
#Diccionario con todas las palabras siguientes a una palabra (ventana de 1) --> dicc_forward
dicc_backward = defaultdict(list)
dicc_forward = defaultdict(list)
for s in check:
    for i,w in enumerate(s):
        if i != 0:
            dicc_backward[w[0]].append((s[i-1][0], s[i-1][1], s[i-1][2], s[i-1][3]))
        else:
            dicc_backward[w[0]].append(('[W.Start]', '[L.Start]', '[T.Start]', '[S.Start]'))
        if i != len(s)-1:
            dicc_forward[w[0]].append((s[i+1][0], s[i+1][1], s[i+1][2], s[i+1][3]))
        else:
            dicc_forward[w[0]].append(('[W.End]', '[T.End]', '[T.End]', '[S.End]'))

In [245]:
words[:2]

[('Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'),
 ('La', 'el', 'DA0FS0', '0')]

In [246]:
count["Plata"]

11

In [247]:
#Creación del diccionario de features
#Recordemos que words es una lista de tuplas del tipo (word,lemma,Pos,synset)
dicc = dict()
vec = dict() #vector de clases
for fst,snd,trd,fth in words:
    features = defaultdict(int)
    #La palabra se encuentra en el dict
    if fst not in dicc and fst not in {'.', ',', ':', ';'} and fst.isalpha() and count[fst] > 1 and fth != 0:
        dicc[fst] = features
        vec[fst] = trd
        features[fth] += 1
        for i in dicc_backward[fst]:
            features[i[0]+"-1"] += 1
            features[i[3]+"-1"] += 1
        for i in dicc_forward[fst]:
            features[i[0]+"+1"] += 1
            features[i[3]+"+1"] += 1
    #La pabra no se encuentra en el dict
    elif fst in dicc:
        if trd not in vec[fst]:
            dicc[fst] = features
            vec[fst] = trd
            for i in dicc_backward[fst]:
                features[i[0]+"-1"] += 1
                features[i[3]+"-1"] += 1
            for i in dicc_forward[fst]:
                features[i[0]+"+1"] += 1
                features[i[3]+"+1"] += 1
        has_it = dicc[fst]
        if fth in has_it:
            has_it[fth] += 1
        else:
            has_it[fth] = 1

In [248]:
#Controlamos que el largo sea consistente
print(len(vec), len(dicc))

7483 7483


In [249]:
#Para remover stopwords
from nltk.corpus import stopwords

In [250]:
matrix = []
vector = []
final_check = []
for key, value in dicc.items():    #preparo la lista de dicts y la lista de chequeo para recuperar el nombre al final
    if key not in stopwords.words('spanish'):
        final_check.append(key)
        matrix.append(value)
for key, value in vec.items():       #vector de PoS
    if key not in stopwords.words('spanish'):
        vector.append(value)

In [251]:
#Transformamos nuestra lista de diccionarios de features en un vector
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
X = v.fit_transform(matrix)

In [252]:
X

<7338x26470 sparse matrix of type '<class 'numpy.float64'>'
	with 102983 stored elements in Compressed Sparse Row format>

In [253]:
k_best = X.shape

In [254]:
#seleccionamos el 70% de los features
kbest = k_best[1]*0.7

In [255]:
kbest

18529.0

In [256]:
#Normalización
import sklearn
X = sklearn.preprocessing.normalize(X)

In [ ]:
##########################################Supervisada, clase: PoS tag ##################################################

In [257]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [258]:
#Método supervisado para feature selection usando la función chi2
Y = SelectKBest(chi2, k=int(kbest)).fit_transform(X, vector)

In [270]:
#Clustering
from sklearn.cluster import KMeans
km = KMeans(n_clusters=80)
km.fit(Y)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=80, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [271]:
labels = km.predict(Y)

In [272]:
#clusters
clusters = defaultdict(set)
for i, label in enumerate(labels):
    clusters[label].add(final_check[i])
print(clusters)

defaultdict(<class 'set'>, {72: {'Nombre', 'Ha', 'Diseñado', 'Texto', 'Debutó', 'La', 'Publicado', 'No', 'Uso', 'Sistema', 'Son', 'Contexto', 'Experiencia', 'Estrenada', 'Fueron', 'Puede', 'Guarda', 'Lista', 'Crea', 'Pertenece', 'Diccionario', 'periodística', 'Quedó', 'Mientras', 'Otras', 'Edición', 'Limita', 'Jugaba', 'Video', 'Mide', 'Algunos', 'Set', 'Dura', 'Vea', 'Ejemplos', 'Colonización', 'Muchos', 'El', 'Convencido', 'Otros', 'Atraído', 'Él', 'Ingresó', 'Nacido', 'Las', 'Tabla', 'Está', 'Estuvo', 'Resultados', 'Siguiendo', 'Habiendo', 'Polígono', 'Organización', 'Web', 'Dicha', 'Dada', 'Ver', 'Se', 'Tienen', 'Escribió', 'Referencia', 'Hizo', 'Obra', 'Colección', 'Cuenta', 'Minas', 'Juega', 'Participó', 'Adaptación', 'Extraña', 'Surgido', 'Siendo', 'Existen', 'Empezó', 'Corresponde', 'Jugó', 'Alberga', 'Perfil', 'Tenía', 'Principales', 'Obtuvo', 'Carretera', 'Este', 'Contiene', 'Estudió', 'Situación', 'Fútbol', 'Ocupa', 'Datos', 'Reacción', 'Aplicaciones', 'Nació', 'Tuvo', 'Pági

In [267]:
print(clusters[97])

{'pastilla', 'píldoras', 'píldora', 'tableta'}


In [265]:
print(clusters[92])

{'debían', 'debió', 'deberían', 'deberá', 'debe', 'deben', 'debería', 'debía'}


In [266]:
print(clusters[88])

{'Bachelor', 'north', 'Art', 'Hour', 'Way', 'east', 'empty', 'implementation', 'west', 'end', 'Because', 'south', 'Sephardim', 'Proceedings', 'Old', 'Journal'}


In [268]:
print(clusters[114])

{'ocupan', 'pobladas', 'ocuparon', 'ocupados', 'vivieron', 'poblados', 'ocupa', 'ocupar', 'ocupó', 'ocupado', 'habitado', 'vivió', 'ocupadas', 'viven'}


In [276]:
print(clusters[19])

{'llamando', 'llamaron', 'llaman', 'llamar', 'llamaban', 'llamado', 'llamó', 'llamadas', 'llamados'}


In [273]:
print(clusters[57])

{'muro', 'muros', 'pared', 'paredes'}


In [274]:
print(clusters[56])

{'largos', 'extensa', 'largas', 'larga', 'extensas'}


In [275]:
print(clusters[77])

{'encontraron', 'observó', 'encontró', 'encontraba', 'descubrieron', 'encontraban', 'encontrar', 'observan', 'observar', 'observa', 'encuentra', 'descubren', 'descubriendo', 'descubre', 'encuentran', 'descubrió', 'encontrará'}


In [277]:
print(clusters[64])

{'usaba', 'utilizando', 'usado', 'usa', 'usada', 'USA', 'usar', 'use', 'utilizaba', 'emplea', 'utilizaron', 'empleado', 'usan', 'usando', 'utilizada', 'usaban', 'usados', 'utilizado', 'utiliza', 'utilizan', 'utilizó', 'utilizar'}
